# 1. Get dynamic pipeline parameters

In [ ]:
# Get folder where the REST downloads were placed
infilefolder = '2021_10_05_07_58_15/'

# Get pipeline name
pipelinename = 'P_Ingest_MelbParkingData'

# Get pipeline run id
loadid = ''

# Get keyvault linked service name
keyvaultlsname = 'Ls_KeyVault_01'

# 2. Load file path variables

In [ ]:
import os
import datetime

token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary

# Primary storage info 
account_name = token_library.getSecretWithLS( keyvaultlsname, "datalakeaccountname")
container_name = 'datalake' # fill in your container name 
relative_path = 'data/lnd/' # fill in your relative folder path 

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path) 
print('Primary storage account path: ' + adls_path) 
load_id = loadid
loaded_on = datetime.datetime.now()
base_path = os.path.join(adls_path, infilefolder)

parkingbay_filepath = os.path.join(base_path, "MelbParkingBayData.json")
print(parkingbay_filepath)
sensors_filepath = os.path.join(base_path, "MelbParkingSensorData.json")
print(sensors_filepath)

# 3. Transform: Standardize

In [ ]:
import ddo_transform.standardize as s

# Retrieve schema
parkingbay_schema = s.get_schema("in_parkingbay_schema")
sensordata_schema = s.get_schema("in_sensordata_schema")

# Read data
parkingbay_sdf = spark.read\
  .schema(parkingbay_schema)\
  .option("badRecordsPath", os.path.join(base_path, "__corrupt", "MelbParkingBayData"))\
  .json(parkingbay_filepath)
sensordata_sdf = spark.read\
  .schema(sensordata_schema)\
  .option("badRecordsPath", os.path.join(base_path, "__corrupt", "MelbParkingSensorData"))\
  .json(sensors_filepath)

# Standardize
t_parkingbay_sdf, t_parkingbay_malformed_sdf = s.standardize_parking_bay(parkingbay_sdf, load_id, loaded_on)
t_sensordata_sdf, t_sensordata_malformed_sdf = s.standardize_sensordata(sensordata_sdf, load_id, loaded_on)

# Insert new rows
t_parkingbay_sdf.write.mode("append").insertInto("interim.parking_bay")
t_sensordata_sdf.write.mode("append").insertInto("interim.sensor")

# Insert bad rows
t_parkingbay_malformed_sdf.write.mode("append").insertInto("malformed.parking_bay")
t_sensordata_malformed_sdf.write.mode("append").insertInto("malformed.sensor")

# Recording record counts for logging purpose
parkingbay_count = t_parkingbay_sdf.count()
sensordata_count = t_sensordata_sdf.count()
parkingbay_malformed_count = t_parkingbay_malformed_sdf.count()
sensordata_malformed_count = t_sensordata_malformed_sdf.count()

# 4. Data Quality
The following uses the [Great Expectations](https://greatexpectations.io/) library. See [Great Expectation Docs](https://docs.greatexpectations.io/docs/) for more info.

**Note**: for simplication purposes, the [Expectation Suite](https://docs.greatexpectations.io/docs/terms/expectation_suite) is created inline. Generally this should be created prior to data pipeline execution, and only loaded during runtime and executed against a data [Batch](https://docs.greatexpectations.io/docs/terms/batch/) via [Checkpoint](https://docs.greatexpectations.io/docs/terms/checkpoint/).

In [ ]:
import pandas as pd
import os
from ruamel import yaml
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    DatasourceConfig,
    FilesystemStoreBackendDefaults,
)
from pyspark.sql import SparkSession, Row

# 0. Create mount point path for spark job
job_id = mssparkutils.env.getJobId()
root_directory=f"/synfs/{job_id}/great_expectations"


# 1. Configure DataContext
# https://docs.greatexpectations.io/docs/terms/data_context
data_context_config = DataContextConfig(
    datasources={
        "parkingbay_data_source": DatasourceConfig(
            class_name="Datasource",
            execution_engine={"class_name": "SparkDFExecutionEngine"},
            data_connectors={
                "parkingbay_data_connector": {
                    "module_name": "great_expectations.datasource.data_connector",
                    "class_name": "RuntimeDataConnector",
                    "batch_identifiers": [
                        "environment",
                        "pipeline_run_id",
                    ],
                }
            }
        )
    },
    store_backend_defaults=FilesystemStoreBackendDefaults(root_directory=root_directory)
)
context = BaseDataContext(project_config=data_context_config)


# 2. Create a BatchRequest based on parkingbay_sdf dataframe.
# https://docs.greatexpectations.io/docs/terms/batch
batch_request = RuntimeBatchRequest(
    datasource_name="parkingbay_data_source",
    data_connector_name="parkingbay_data_connector",
    data_asset_name="paringbaydataaset",  # This can be anything that identifies this data_asset for you
    batch_identifiers={
        "environment": "stage",
        "pipeline_run_id": "pipeline_run_id",
    },
    runtime_parameters={"batch_data": parkingbay_sdf},  # Your dataframe goes here
)


# 3. Define Expecation Suite and corresponding Data Expectations
# https://docs.greatexpectations.io/docs/terms/expectation_suite
expectation_suite_name = "parkingbay_data_exception_suite_basic"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)
# Add Validatons to suite
# Check available expectations: validator.list_available_expectation_types()
# https://legacy.docs.greatexpectations.io/en/latest/autoapi/great_expectations/expectations/index.html
# https://legacy.docs.greatexpectations.io/en/latest/reference/core_concepts/expectations/standard_arguments.html#meta
validator.expect_column_values_to_not_be_null(column="meter_id")
validator.expect_column_values_to_not_be_null(column="marker_id")
validator.expect_column_values_to_be_of_type(column="rd_seg_dsc", type_="StringType")
validator.expect_column_values_to_be_of_type(column="rd_seg_id", type_="IntegerType")
# validator.validate() # To run run validations without checkpoint
validator.save_expectation_suite(discard_failed_expectations=False)


# 4. Configure a checkpoint and run Expectation suite using checkpoint
# https://docs.greatexpectations.io/docs/terms/checkpoint
my_checkpoint_name = "Parkingbay Data DQ"
checkpoint_config = {
    "name": my_checkpoint_name,
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
}
my_checkpoint = context.test_yaml_config(yaml.dump(checkpoint_config))
context.add_or_update_checkpoint(**checkpoint_config)
# Run Checkpoint passing in expectation suite.
checkpoint_result = context.run_checkpoint(
    checkpoint_name=my_checkpoint_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name,
        }
    ],
)


# 5. Observability: Logging to Azure Application Insights using OpenCensus Library

In [ ]:
import logging
import os
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.azure.log_exporter import AzureEventHandler
from datetime import datetime

# Getting Application Insights instrumentation key
appi_key = token_library.getSecretWithLS(keyvaultlsname,"applicationInsightsKey")

# Enable App Insights
aiLogger = logging.getLogger(__name__)
aiLogger.addHandler(AzureEventHandler(connection_string = 'InstrumentationKey=' + appi_key))

aiLogger.setLevel(logging.INFO)

aiLogger.info("Standardize (ai): Started at " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
properties = {"custom_dimensions": {"pipeline": pipelinename, "run_id": loadid, "parkingbay_count": parkingbay_count, "sensordata_count": sensordata_count, "parkingbay_malformed_count": parkingbay_malformed_count, "sensordata_malformed_count": sensordata_malformed_count}}
aiLogger.info("Standardize (ai): Completed at " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"), extra=properties)

# To query this log, go to the Azure Monitor and run the following kusto query (Scope: Application Insights instance):
#customEvents
#| order by timestamp desc
#| project timestamp, appName, name,
#    pipelineName             = customDimensions.pipeline,
#    pipelineRunId            = customDimensions.run_id,
#    parkingbayCount          = customDimensions.parkingbay_count,
#    sensordataCount          = customDimensions.sensordata_count,
#    parkingbayMalformedCount = customDimensions.parkingbay_malformed_count,
#    sensordataMalformedCount = customDimensions.sensordata_malformed_count,
#    dimParkingbayCount       = customDimensions.new_parkingbay_count


# 6. Observability: Logging to Log Analytics workspace using log4j

In [ ]:
import logging
import sys

# Enable Log Analytics using log4j
log4jLogger = sc._jvm.org.apache.log4j
logger = log4jLogger.LogManager.getLogger("ParkingSensorLogs")

def log(msg = ''):
    env = mssparkutils.env
    formatted_msg = f'Standardize (log4j): {msg}~{pipelinename}~{env.getJobId()}~{env.getPoolName()}~{env.getWorkspaceName()}~{env.getUserId()}'
    logger.info(formatted_msg)

log("Started at " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

log(f'parkingbay_count: {parkingbay_count}')
log(f'sensordata_count: {sensordata_count}')
log(f'parkingbay_malformed_count: {parkingbay_malformed_count}')
log(f'sensordata_malformed_count: {sensordata_malformed_count}')

log("Completed at " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# To query this log, go to the log analytics workspace and run the following kusto query (Scope: Log Analytics Workspace):
#SparkLoggingEvent_CL
#| where logger_name_s == "ParkingSensorLogs"
#| order by TimeGenerated desc
#| project TimeGenerated, workspaceName_s, Level,
#    message         = split(Message, '~', 0),
#    pipelineName    = split(Message, '~', 1),
#    jobId           = split(Message, '~', 2),
#    SparkPoolName   = split(Message, '~', 3),
#    UserId          = split(Message, '~', 5)